In [1]:
import numpy as np
from typing import Tuple

np.random.seed(10)
ndarray = np.ndarray

# Create simulated data

In [2]:
N = 100
d = 10
wbx_mu = 0
w_sd = 2
bx_sd = 10
noise_mu = 0
noise_sd = .5

# 10 features plus bias
w_true = np.random.normal(loc=wbx_mu, scale=w_sd, size=(d,))
b_true = np.random.normal(loc=wbx_mu, scale=bx_sd)

# Generate training data and labels (add some noise to labels)
X_train = np.random.normal(loc=wbx_mu, scale=bx_sd, size=(N, d))
y_train = X_train @ w_true + b_true + np.random.normal(loc=noise_mu, scale=noise_sd, size=(X_train.shape[0],))
# Create some outliers
outlier_idx = np.random.randint(y_train.shape[0], size=5)
y_train[outlier_idx] += np.random.normal(loc=noise_mu, scale=noise_sd * 10, size=(outlier_idx.shape[0]))

# Create a linear regression model

In [3]:
from numpy.linalg import solve, norm

L2 = "l2"
L1 = "l1"
HUBER = "huber"
NORM_EQ = "normal_equations"
GD = "gradient_descent"
ZEROS = "zeros"
RAND = "random"


def norm_eq_se(X, y) -> Tuple[ndarray, float]:
    res = solve(X.T @ X, X.T @ y)
    return res[:-1], res[-1]


def grad(X, y, loss_f, alpha=0.0001, tolerance=0.001, max_t=1000, init="zeros") -> ndarray:
    # setup
    if init == ZEROS:
        w = np.zeros(shape=(X.shape[1],))
    elif init == RAND:
        w = np.random.normal(loc=0, scale=1, size=(X.shape[1],))
    else:
        raise NotImplementedError(f"Desired initialization of w ({init}) is not implemented.")

    
    if loss_f == L2:
        X_t_y = X.T @ y
        grad_f = lambda w: X.T @ (X @ w) - X_t_y
    elif loss_f == L1:
        grad_f = lambda w: X.T @ np.sign(X @ w - y)
    elif loss_f == HUBER:
        grad_f = lambda w: X.T @ h_prime(X @ w - y)
    else:
        raise NotImplementedError(f"Desired loss ({loss_f}) is not implemented.")
    
    # run gradient descent
    g = grad_f(w)
    loss = norm(g)
    t = 0
    while loss > tolerance and t < max_t:
        g = grad_f(w)
        loss = norm(g)
        w -= alpha * g
        t += 1
        
    return w[:-1], w[-1]

@np.vectorize
def h_prime(z) -> float:
    z_abs = np.abs(z)
    if z_abs <= 1:
        return z
    else:
        return np.sign(z)
    
    
class LinearRegression:
    def __init__(self):
        self.w: ndarray = np.array([])
        self.b: float = 0.0
            
    @classmethod
    def from_fit(cls, X, y, loss, optimization, **kwargs):
        lr = cls()
        lr.fit(X, y, loss, optimization, **kwargs)
        return lr
    
    def fit(self, X, y, loss, optimization, **kwargs) -> None:
        # take bias into account
        X = np.concatenate((X, np.ones(shape=(X.shape[0], 1))), axis=1)
        
        if loss == L2 and optimization == NORM_EQ:
            self.w, self.b = norm_eq_se(X, y, **kwargs)
        elif optimization == GD:
            self.w, self.b = grad(X, y, loss_f=loss, **kwargs)
        
    def predict(self, X) -> ndarray:
        return X @ self.w + self.b

# Create some new data

In [4]:
X_test = np.random.normal(loc=wbx_mu, scale=bx_sd, size=(N, d))
# Add noise
y_test = X_test @ w_true + b_true + np.random.normal(loc=noise_mu, scale=noise_sd, size=(N,))
# Create some outliers
outlier_idx = np.random.randint(y_test.shape[0], size=10)
y_test[outlier_idx] += np.random.normal(loc=noise_mu, scale=noise_sd * 5, size=(outlier_idx.shape[0]))

# Check model using different approaches

We check to see that the parameters of our model match those that were used to create the simulated data.

In [5]:
TOL = 1e-5

def check_params(w, b, lr, tol):
    print((f"Results:\n"
           f"l1  |w_true - w_comp| = {norm((w - lr.w), ord=1)}\n"
           f"l1  |b_true - b_comp| = {np.abs(b - lr.b)}"))

def check_preds(true, pred, tol):
    print(f"l1  |y_true - y_pred| = {norm((true - pred), ord=1)}")

## 1. L2 (Normal Equations)

In [6]:
lr_n = LinearRegression.from_fit(X_train,
                                 y_train,
                                 loss="l2",
                                 optimization="normal_equations")
y_pred = lr_n.predict(X_test)
check_params(w_true, b_true, lr_n, TOL)
check_preds(y_test, lr_n.predict(X_test), TOL)

Results:
l1  |w_true - w_comp| = 0.14743165590808693
l1  |b_true - b_comp| = 0.08880078317296825
l1  |y_true - y_pred| = 74.5912786083365


## 2. L2 (Gradient Descent)

In [7]:
lr_g_l2 = LinearRegression.from_fit(X_train,
                                    y_train,
                                    loss="l2",
                                    optimization="gradient_descent",
                                    tolerance=TOL,
                                    max_t=int(1e4),
                                    init="zeros")
y_pred = lr_g_l2.predict(X_test)
check_params(w_true, b_true, lr_g_l2, TOL)
check_preds(y_test, lr_g_l2.predict(X_test), TOL)

Results:
l1  |w_true - w_comp| = 0.14743165855938828
l1  |b_true - b_comp| = 0.0888006806331969
l1  |y_true - y_pred| = 74.59127794995652


## 3. L1 (Gradient Descent)

In [8]:
lr_g_l1 = LinearRegression.from_fit(X_train,
                                    y_train,
                                    loss="l1",
                                    optimization="gradient_descent",
                                    tolerance=TOL,
                                    max_t=int(1e4),
                                    init="zeros")
y_pred = lr_g_l2.predict(X_test)
check_params(w_true, b_true, lr_g_l1, TOL)
check_preds(y_test, lr_g_l1.predict(X_test), TOL)

Results:
l1  |w_true - w_comp| = 0.10323016377077289
l1  |b_true - b_comp| = 0.06593810046406468
l1  |y_true - y_pred| = 62.76632579907822


## 4. Huber function (Gradient Descent)

This is a "smoothed out" $L1$ norm.

Hubler function (and derivative):

$
h(z) \equiv
\begin{cases}
\frac{1}{2}z^2 & |z| \leq 1 \\
|z|-\frac{1}{2} & |z| \gt 1 \\
\end{cases}
$

$
h'(z) \equiv
\begin{cases}
z & |z| \leq 1 \\
sign(z) & |z| \gt 1 \\
\end{cases}
$

In [9]:
lr_g_h = LinearRegression.from_fit(X_train,
                                    y_train,
                                    loss="huber",
                                    optimization="gradient_descent",
                                    tolerance=TOL,
                                    max_t=int(1e4),
                                    init="zeros")
y_pred = lr_g_h.predict(X_test)
check_params(w_true, b_true, lr_g_h, TOL)
check_preds(y_test, lr_g_h.predict(X_test), TOL)

Results:
l1  |w_true - w_comp| = 0.05741787448924967
l1  |b_true - b_comp| = 0.013442334224661678
l1  |y_true - y_pred| = 58.785678513579754
